# Requires HF_TOKEN & GOOGLE_API_KEY

In [13]:
!pip install sacremoses -q

import time
from transformers import pipeline
import google.generativeai as genai
from google.colab import userdata

In [14]:
user_text = """Samsung smartphones are often considered a better choice than iPhones for various reasons.
They offer a wider range of models, providing options for different budgets and user needs.
Samsung devices typically come with more advanced hardware features, such as higher-resolution displays, expandable storage, and larger battery capacities.
The Android ecosystem also allows for greater customization and flexibility compared to Apple's iOS.
Overall, Samsung's innovation and variety make it a strong competitor and, for many users, a more suitable option than the iPhone."""

Summarization

Part A: Using Open-Source Transformers (Hugging Face)

In [15]:
# Load the summarization pipeline
summarizer = pipeline("summarization", model="google-t5/t5-small")

text = f"summarize: {user_text}"

start_time = time.time()
open_source_result = summarizer(text, max_length=50, min_length=10, do_sample=False)
open_source_latency = time.time() - start_time

# Display result
print("Open-Source Summary:", open_source_result[0]['summary_text'])
print("Latency:", open_source_latency, "seconds")

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Open-Source Summary: smartphones offer a wider range of models, providing options for different budgets and user needs . the Android ecosystem also allows for greater customization and flexibility compared to the iPhone .
Latency: 5.726796627044678 seconds


Part B: Using Proprietary Transformers (Google Gemini API)

In [16]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Configure Gemini Model
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash')

prompt = f"Summarize the following text in a concise manner: {user_text}"

start_time = time.time()
response = model.generate_content(prompt)
proprietary_result = response.text
proprietary_latency = time.time() - start_time

# Display result
print("Proprietary Summary:", proprietary_result)
print("Latency:", proprietary_latency, "seconds")

Proprietary Summary: Samsung smartphones are often preferred over iPhones due to their wider range of models for different budgets and needs, more advanced hardware features (like high-resolution displays, expandable storage, and larger batteries), and the greater customization offered by the Android ecosystem. Their innovation and variety make them a strong and often more suitable choice for many users.
Latency: 6.19415807723999 seconds


Text Classification

Part A: Using Open-Source Transformers (Hugging Face)

In [17]:
# Load zero-shot classification pipeline for text classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["technology", "government policy", "education", "economy", "health"]

start = time.time()
result_hf = classifier(user_text, candidate_labels)
end = time.time()

print("---- Hugging Face Text Classification ----")
print("Top Labels:", result_hf['labels'][:3])
print("Scores:", result_hf['scores'][:3])
print("\nLatency:", round(end - start, 2), "seconds")

Device set to use cpu


---- Hugging Face Text Classification ----
Top Labels: ['technology', 'economy', 'health']
Scores: [0.8105211853981018, 0.11234080791473389, 0.04977939650416374]

Latency: 33.92 seconds


Part B: Using Proprietary Transformers (Google Gemini API)

In [18]:
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash')

start = time.time()
response = model.generate_content(
    f"Classify the following text into one primary category from these options: technology, government policy, education, economy, health. "
    f"Provide top 3 categories:\n\n{user_text}"
)
end = time.time()

print("---- Gemini Text Classification ----")
print(response.text)
print("\nLatency:", round(end - start, 2), "seconds")

---- Gemini Text Classification ----
Primary Category: **Technology**

Top 3 Categories:

1.  **Technology:** The text is fundamentally about technological products (smartphones), their hardware features (displays, storage, battery), software ecosystems (Android, iOS), innovation, and user experience.
2.  **Economy:** The text discusses market aspects such as "different budgets," "strong competitor," "variety" (implying consumer choice and market segmentation), and the competitive landscape between major brands like Samsung and Apple.
3.  **Education:** While not explicitly mentioned in the text, smartphones are ubiquitous tools in modern education (e.g., for research, learning apps, communication). This is a very tangential connection for this specific text, but among the remaining options, it's a slightly less distant association with the *product category* itself than government policy or health, if one must force a third from the given list. (It's important to note that the text it

Sentiment Analysis


Part A: Using Open-Source Transformers (Hugging Face)

In [19]:
# Load sentiment analysis model
classifier = pipeline("sentiment-analysis")

start = time.time()
result_hf = classifier(user_text)
end = time.time()

print("---- Hugging Face Sentiment ----")
print(result_hf)
print("\nLatency:", round(end - start, 2), "seconds")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


---- Hugging Face Sentiment ----
[{'label': 'POSITIVE', 'score': 0.9969255328178406}]

Latency: 0.83 seconds


Part B: Using Proprietary Transformers (Google Gemini API)

In [20]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)

start = time.time()
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"Perform sentiment analysis on the following text. "
             f"Classify it as Positive, Negative, or Neutral, and briefly explain:\n\n{user_text}"
)
end = time.time()

print("---- Gemini Sentiment ----")
print(response.text)
print("\nLatency:", round(end - start, 2), "seconds")


---- Gemini Sentiment ----
**Classification:** Positive

**Explanation:** The text consistently praises Samsung smartphones, highlighting their advantages with phrases like "better choice," "wider range of models," "more advanced hardware features," "greater customization and flexibility," "innovation and variety," and being a "strong competitor" and "more suitable option." There are no negative or neutral terms used to describe Samsung's offerings; all points emphasize their superiority or desirable qualities.

Latency: 4.94 seconds


Translation - English to Hindi

Part A: Using Open-Source Transformers (Hugging Face)

In [21]:
translator = pipeline("translation_en_to_hi", model="Helsinki-NLP/opus-mt-en-hi")

start = time.time()
result_hf = translator(user_text, max_length=200)
end = time.time()

print("---- Hugging Face Translation (English → Hindi) ----")
print(result_hf[0]['translation_text'])
print("\nLatency:", round(end - start, 2), "seconds")

Device set to use cpu


---- Hugging Face Translation (English → Hindi) ----
सैम हंग स्मार्टफोनों को विभिन्न कारणों से एक बेहतर विकल्प माना जाता है। वे मॉडलों की एक विस्तृत सीमा प्रदान करते हैं, विभिन्न बजट और उपयोगकर्ता की जरूरतों के लिए विकल्प प्रदान करते हैं। सैमंग उपकरणों आम तौर पर अधिक उन्नत हार्डवेयर सुविधाओं, जैसे कि अधिक विस्तृत हार्डवेयर सुविधाओं, विकास, विकास, और बड़ी बैटरी के साथ आता है। और इससे भी अधिक जटिलता की अनुमति मिलती है। और मैं पहली बार इग्नेज़ की तुलना में इडिम्पलिटी और अधिक जटिल और अधिक महत्वपूर्ण उपभोक्ताओं की तुलना में। और यह बहुत से अधिक महत्वपूर्ण विकल्प बनाने की जरूरत है। और यह वास्तव में, मैं वास्तव में बहुत से अधिक महत्वपूर्ण मॉडलों के लिए एक बहुत से अधिक महत्वपूर्ण विकल्प हैं।

Latency: 11.5 seconds


Part B: Using Proprietary Transformers (Google Gemini API)

In [22]:
start = time.time()
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"Translate the following English text into Hindi:\n\n{user_text}"
)
end = time.time()

print("---- Gemini Translation (English → Hindi) ----")
print(response.text)
print("\nLatency:", round(end - start, 2), "seconds")

---- Gemini Translation (English → Hindi) ----
सैमसंग स्मार्टफोन को अक्सर आईफोन की तुलना में कई कारणों से एक बेहतर विकल्प माना जाता है।

ये मॉडलों की एक विस्तृत श्रृंखला पेश करते हैं, जो विभिन्न बजटों और उपयोगकर्ता की ज़रूरतों के लिए विकल्प प्रदान करते हैं। सैमसंग डिवाइस आमतौर पर अधिक उन्नत हार्डवेयर सुविधाओं के साथ आते हैं, जैसे कि उच्च-रिज़ॉल्यूशन वाले डिस्प्ले, विस्तार योग्य स्टोरेज और बड़ी बैटरी क्षमताएं। एंड्रॉइड इकोसिस्टम भी एप्पल के आईओएस की तुलना में अधिक अनुकूलन (कस्टमाइजेशन) और लचीलेपन की अनुमति देता है।

कुल मिलाकर, सैमसंग का नवाचार और विविधता इसे एक मजबूत प्रतियोगी बनाते हैं और कई उपयोगकर्ताओं के लिए, आईफोन की तुलना में एक अधिक उपयुक्त विकल्प हैं।

Latency: 8.43 seconds
